In [1]:
import pytesseract
from PIL import Image
import cv2 
import openai
import os
import getpass 
import tqdm 
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import pickle
import ast
import torch
import json 




import datasets
from StringUtils import CorrectDigits, AssignNERTags, AssignLabel
from OCRUtils import apply_tesseract,DrawBoundBoxes,denormalize_box, ProcessImage
from transformers import LayoutLMv3ForTokenClassification, LayoutLMv3Tokenizer, LayoutLMv3FeatureExtractor, LayoutLMv3TokenizerFast,LayoutLMv3Processor
from PIL import Image, ImageDraw, ImageFont
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset

from DataLoader import TokenClassificationDataset

from model import ModelModule
from sklearn.model_selection import train_test_split



In [3]:
DatasetSROIE2019V1 = pd.read_pickle('DatasetSROIE2019V3')

In [6]:
ner_tags_SROIE2019 = []
indexes_to_drop = []
image_column = []

for indx ,row in tqdm(DatasetSROIE2019V1.iterrows()):
    words = row['words'] 
    boxes = row['boxes']
    entities = row['entities']


    ner_tags = AssignNERTags(words=words,entities=row['entities'],boxes=boxes)

    ner_tags_SROIE2019.append(ner_tags)
    
    filename = row['filename']
    folder_path = './SROIE2019/train/img'
    image_path = os.path.join(folder_path, filename)
    image = Image.open(image_path)
    image = image.resize((600,1000))

    image_column.append(image)

DatasetSROIE2019V1['ner_tags'] = ner_tags_SROIE2019
DatasetSROIE2019V1['image'] = image_column

0it [00:00, ?it/s]

626it [00:59, 10.59it/s]


In [7]:
for indx ,row in tqdm(DatasetSROIE2019V1.iterrows()):
    image = row['image']
    image_array = np.array(image)
    ndim = image_array.ndim
    if ndim < 3 :
        print(indx, f'has only {ndim} dimensions ')
        indexes_to_drop.append(indx)

DatasetSROIE2019V1.drop(index = indexes_to_drop, inplace= True)


492it [00:01, 348.97it/s]

459 has only 2 dimensions 
460 has only 2 dimensions 
461 has only 2 dimensions 


626it [00:01, 410.43it/s]


In [8]:
DatasetSROIE2019V1 = DatasetSROIE2019V1.reset_index()

In [9]:
# # load model and processor from huggingface hub
from transformers import LayoutLMv3ImageProcessor
feature_extractor = LayoutLMv3ImageProcessor(apply_ocr= False, do_normalize= True, do_resize= True)
tokenizer = LayoutLMv3TokenizerFast.from_pretrained("microsoft/layoutlmv3-base", max_length = 520)
processor = LayoutLMv3Processor(feature_extractor,tokenizer)
# model = LayoutLMv3ForTokenClassification.from_pretrained('microsoft/layoutlmv3-base', num_labels = 6)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [26]:
class TokenClassificationDataset(Dataset):
 
    def __init__(self, Dataframe, processor, folder_path_images):
        self.Dataframe = Dataframe
        self.processor = processor
        self.folder_path_images = folder_path_images
        self.receiptClasses = self.ReceiptClasses()
 
    def __len__(self):
        return len(self.Dataframe)
 
    def __getitem__(self, item):
        if isinstance(item, list):  # Check if item is a list
            images = []
            labels = []
            words = []
            boxes = []
            for idx in item:
                image_file = self.Dataframe['filename'][idx]
                image_path = os.path.join(self.folder_path_images, image_file)
                with Image.open(image_path).convert("RGB") as image:

                    image = image.resize((600,1000))
                    images.append(image)
                label = self.Dataframe['ner_tags'][idx]
                labels.append(self.ConvertLabel2id(labels=label))
                words.append(self.Dataframe['words'][idx])
                boxes.append(self.Dataframe['boxes'][idx])

                

        else:
            image_file = self.Dataframe['filename'][item]
            image_path = os.path.join(self.folder_path_images, image_file)
            with Image.open(image_path).convert("RGB") as image:

                images = image.resize((600,1000))
            labels = self.Dataframe['ner_tags'][item]
            labels = self.ConvertLabel2id(labels=labels)
                # extracting words list, boxes list and labels (for TokenClassificatiion Task) list        
            words = self.Dataframe['words'][item]
            boxes = self.Dataframe['boxes'][item]
            

        # processing the data    
        encoding = self.processor(
            images,
            words,
            boxes=boxes,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        

 
        return dict(
            input_ids=encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            bbox=encoding["bbox"].flatten(end_dim=1),
            pixel_values=encoding["pixel_values"].flatten(end_dim=1),
            labels=torch.tensor(encoding['labels'][0], dtype=torch.long)
        )
    
    
    # convert string labels to ids
    def ConvertLabel2id(self,labels : list[str]) :
        return [self.receiptClasses[label] for label in labels]
    
    # defines the dictionary of classes to id conversion 
    def ReceiptClasses(self):
        return {'O' : 0, 'COMPANY' : 1, 'LOCATION' : 2, 'DATE' : 3, 'TOTAL': 4, 'TAX': 5, 'CURRENCY' : 6}

In [12]:
RECEIPT_CLASSES = ['O', 'COMPANY' , 'LOCATION' , 'DATE' , 'TOTAL', 'TAX',  'CURRENCY']
label2id = {v: k for k,v in enumerate(RECEIPT_CLASSES)}

DatasetSROIE2019V1['ner_tags'] = DatasetSROIE2019V1['ner_tags'].apply(lambda x : [label2id[label] for label in x] )

KeyError: 0

In [22]:
from torch.utils.data import Dataset
class TokenClassificationDataset(Dataset):
 
    def __init__(self, Dataframe, processor):
        self.Dataframe = Dataframe
        self.processor = processor
 
    def __len__(self):
        return len(self.Dataframe)
 
    def __getitem__(self, item):

        # extracting words list, boxes list and labels (for TokenClassificatiion Task) list        
        words = self.Dataframe['words'][item]
        boxes = self.Dataframe['boxes'][item]
        labels = self.Dataframe['ner_tags'][item]
        image = self.Dataframe['image'][item]

        # processing the data    
        encoding = self.processor(
            image,
            words,
            boxes=boxes,
            word_labels = labels,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )


        print(item)
        return dict(
            input_ids=encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            bbox=encoding["bbox"].flatten(end_dim = 1),
            pixel_values=encoding["pixel_values"].flatten(end_dim = 1),
            labels = encoding['labels'].flatten()
        )


In [59]:
from sklearn.model_selection import train_test_split

folder_path = './SROIE2019/train/img'

data_train, data_test = train_test_split(DatasetSROIE2019V1,test_size= 0.2)


data_train.reset_index(drop=True, inplace=True)
data_test.reset_index(drop=True, inplace=True)

train_dataset = TokenClassificationDataset(data_train, processor)
eval_dataset = TokenClassificationDataset(data_test, processor)

loader_train = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
)

loader_eval = DataLoader(
    eval_dataset,
    batch_size=4,
    shuffle=False,
)

In [57]:
for i, data in enumerate(train_dataset):
    print(i)

KeyError: 498

In [60]:
for batch in loader_train: 
    print('success')

In [9]:
id2label = {k: v for k,v in enumerate(RECEIPT_CLASSES)}
model_module = ModelModule(len(RECEIPT_CLASSES))

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

Reusing TensorBoard on port 6006 (pid 21856), started 3 days, 0:05:04 ago. (Use '!kill 21856' to kill it.)

In [10]:
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl
 
model_checkpoint = ModelCheckpoint(
    filename="{epoch}-{step}-{val_loss:.4f}", save_last=True, save_top_k=3, monitor="val_loss", mode="min"
)
 
trainer = pl.Trainer(
    accelerator="gpu",
    precision=16,
    devices=1,
    max_epochs=5,
    callbacks=[
        model_checkpoint
    ],
)
     


c:\Users\Natsu\Anaconda3\lib\site-packages\lightning_fabric\connector.py:563: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [43]:
model = LayoutLMv3ForTokenClassification.from_pretrained(
            "microsoft/layoutlmv3-base",
            num_labels=len(RECEIPT_CLASSES)
        )



Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
for batch in loader_train:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    bbox = batch['bbox']
    pixel_values = batch['pixel_values']
    labels = batch['labels']
    
    outputs = model(
            input_ids,
            attention_mask=attention_mask,
            bbox=bbox,
            pixel_values=pixel_values,
            labels=labels
        )
    
    print(outputs.shape)
    break

In [42]:
trainer.fit(model_module, loader_train, loader_eval)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                             | Params
--------------------------------------------------------------------
0 | model          | LayoutLMv3ForTokenClassification | 125 M 
1 | train_accuracy | MulticlassAccuracy               | 0     
2 | val_accuracy   | MulticlassAccuracy               | 0     
--------------------------------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
501.329   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Natsu\Anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
C:\Users\Natsu\AppData\Local\Temp\ipykernel_13620\4178943587.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels=torch.tensor(encoding['labels'][0], dtype=torch.long)
c:\Users\Natsu\Anaconda3\lib\site-packages\transformers\modeling_utils.py:1051: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


ValueError: If `preds` have one dimension more than `target`, `preds.shape[1]` should be equal to number of classes.

In [22]:
import model
import importlib
importlib.reload(model)
from model import *

In [38]:
%reload_ext autoreload
%autoreload 2